In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib.font_manager import FontProperties
from matplotlib.backends.backend_pdf import PdfPages
from math import *
import pickle

from pathlib import Path
from tqdm.auto import tqdm

# coding: utf-8
import time
# import cProfile
# from mpl_toolkits.mplot3d import Axes3D
import random
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn import metrics

In [2]:
def Value_Func_original(estados, RTT, ro, cost_measure, max_slots, N):
    # RTT es el vector de valores que va tomando el camino elegido en cada time_slot
    # ro factor de descuento
    # cost_measure vector de costo de medida en cada time_slot
    # max_slots es que tanto considero el futuro
    # en V guardo el promedio de la value function del estado que supero N visitas, y cuantas veces pase por ese estado
    V={}
    val=np.zeros(len(RTT)-max_slots)
    vector_ro=(np.ones(max_slots)*ro)**np.arange(max_slots)
    RTT=np.array(RTT)
    cost_measure=np.array(cost_measure)
    # print(max_slots)
    # print(vector_ro)
    # print(RTT)
    # print(cost_measure)
    for i in range(len(RTT)-max_slots):
        # print(i)
        val[i]=sum((RTT[i:i+max_slots]+cost_measure[i:i+max_slots])*vector_ro.T)

        if estados[i] in V:
            V[estados[i]]=[(val[i]+V[estados[i]][0]*V[estados[i]][1])/(V[estados[i][1]]+1), V[estados[i]][1]+1]
        # print(RTT[i:i+max_slots]+cost_measure[i:i+max_slots])
        else:
            V[estados[i]]=[val[i], 1]
    return V


In [3]:
def Value_Func(estados, RTT, ro, cost_measure, max_slots):
    # RTT es el vector de valores que va tomando el camino elegido en cada time_slot
    # ro factor de descuento
    # cost_measure vector de costo de medida en cada time_slot
    # max_slots es que tanto considero el futuro
    # en V guardo el promedio de la value function del estado que supero N visitas, y cuantas veces pase por ese estado
    V={}
    val=np.zeros(len(RTT)-max_slots)
    vector_ro=(np.ones(max_slots)*ro)**np.arange(max_slots)
    RTT=np.array(RTT)
    cost_measure=np.array(cost_measure)
    # print(max_slots)
    # print(vector_ro)
    # print(RTT)
    # print(cost_measure)
    for i in range(len(RTT)-max_slots):
        # print(i)
        val[i]=sum((RTT[i:i+max_slots]+cost_measure[i:i+max_slots])*vector_ro.T)
        # print(val[i])
        if estados[i] in V:
            V[estados[i]]=[(val[i]+V[estados[i]][0]*V[estados[i]][1])/(V[estados[i]][1]+1), V[estados[i]][1]+1]
            # print(V[estados[i]])
        # print(RTT[i:i+max_slots]+cost_measure[i:i+max_slots])
        else:
            V[estados[i]]=[val[i], 1]
    return V


In [4]:
def get_Markov_delay(P,M,current_state):
	#SIMULATION OF THE MARKOV ROAD
	states_MR=np.array([])
	Pcumsum = np.cumsum(P,1)
	# esta de abajo no la precisamo'
	states = np.array([current_state])
	for t in range(0,M):
		U = np.random.random()
		pnext = Pcumsum[current_state,:]
		current_state = np.sum(U>pnext)
		states_MR = np.append(states_MR, current_state)
	return states_MR

In [5]:
def series_de_markov(largo, P, niveles):
    serie_markov_1=get_Markov_delay(P, largo, 1)
    serie_markov_1_mod=np.zeros(len(serie_markov_1))
    for i in range(largo):
        for j in range(len(niveles)):
            if int(serie_markov_1[i])==j:
                serie_markov_1_mod[i]=niveles[j]
    return serie_markov_1, serie_markov_1_mod
